# Assignment 2A

**Name**:    Rohit Raj                           </br>
**Roll No.**: 210874

In [207]:
# import
import gymnasium as gym
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import random

In [208]:
'''
Represents a Stochastic Maze problem Gym Environment which provides a Fully observable
MDP
'''
class StochasticMazeEnv(gym.Env):
    '''
    StochasticMazeEnv represents the Gym Environment for the Stochastic Maze environment
    States : [0,1,2,3,4,5,6,7,8,9,10,11]
    Actions : ["Left":0, "Up":1, "Right":2, "Down":3]
    '''
    metadata = {'render.modes': ['human']}

    def __init__(self,initial_state=0,no_states=12,no_actions=4):
        '''
        Constructor for the StochasticMazeEnv class

        Args:
            initial_state : starting state of the agent
            no_states : The no. of possible states which is 12
            no_actions : The no. of possible actions which is 4
            
        '''
        self.initial_state = initial_state
        self.state = self.initial_state
        self.nA = no_actions
        self.nS = no_states
        self.actions_dict = {"L":0, "U":1, "R":2, "D":3}
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        self.reset()

    def reset(self):
        '''
        Resets the environment
        Returns:
            observations containing player's current state
        '''
        self.state = self.initial_state
        return self.get_obs()

    def get_obs(self):
        '''
        Returns the player's state as the observation of the environment
        '''
        return (self.state)

    def render(self, mode='human'):
        '''
        Renders the environment
        '''
        print("Current state: {}".format(self.state))

    def sample_action(self):
        '''
        Samples and returns a random action from the action space
        '''
        return random.randint(0, self.nA)
    def P(self):
        '''
        Defines and returns the probabilty transition matrix which is in the form of a nested dictionary
        '''
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        return self.prob_dynamics


    def step(self, action):
        '''
        Performs the given action
        Args:
            action : action from the action_space to be taking in the environment
        Returns:
            observation - returns current state
            reward - reward obtained after taking the given action
            done - True if the episode is complete else False
        '''
        if action >= self.nA:
            action = self.nA-1

        index = np.random.choice(3,1,p=[0.8,0.1,0.1])[0]

        dynamics_tuple = self.prob_dynamics[self.state][action][index]
        

        self.state = dynamics_tuple[1]
        
                

        return self.state, dynamics_tuple[2], dynamics_tuple[3]

In [209]:
env = StochasticMazeEnv()
env.reset()
num_states = env.nS
num_actions = env.nA

## Test Cases for checking the environment 

## Example: Random Policy

In [210]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    rand_action = np.random.choice(4,1)[0]  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(rand_action)
    
    total_reward += reward

    print("  ", rand_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   0 	   0 	 -0.01 	   False
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 4
Final Reward: 0.97


### The random policy takes large number of steps to reach some terminal state which should be much higher than the number of the steps taken by a all 'Right' policy

## Example: Right Policy

In [211]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    right_action = 2  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(right_action)
    
    total_reward += reward

    print("  ", right_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 3
Final Reward: 0.98


### The right policy most of the time reaches the goal state in just 3 steps which is expected

***

## Q1. Find an optimal policy to navigate the given environment using Policy Iteration 

In [212]:
import numpy as np

prob_dynamics=env.prob_dynamics
actions_dict = env.actions_dict

# Policy iteration function
def policy_iteration(prob_dynamics, actions_dict, discount_factor=0.4, theta=0.00001,num_iteration_PI=0):
    num_states = 12
    num_actions = 4
    
    # Initialize a random policy
    policy = np.zeros((num_states, num_actions)) # policy[a][b] denotes the probability of taking action b at the state a 
    while True:
        # Policy Evaluation
        num_iteration_PI+=1
        V = policy_evaluation(prob_dynamics, policy, discount_factor, theta) # It will return the value function of all the states a/c to the policy 
        
        policy_stable = True
        # Policy Improvement
        for state in range(num_states):   
            old_action = np.argmax(policy[state])
            if state==5:
                continue
            
            # Compute action values for the current state
            action_values = np.zeros(num_actions)
            for action in range(num_actions):
                for prob, next_state, reward, is_terminal in prob_dynamics[state][action]:
                    action_values[action] += prob * (reward + discount_factor * V[next_state])

            # Greedily update the policy
            best_action = np.argmax(action_values)
            policy[state] = np.eye(num_actions)[best_action]
            
            # Check if policy is stable
            if old_action != best_action:
                policy_stable = False
        
        # If policy did not change, converged
        if policy_stable:
            break
            
    return policy, V,num_iteration_PI

# Policy evaluation function
def policy_evaluation(prob_dynamics, policy, discount_factor, theta):
    num_states=12
    V = np.zeros(num_states) # initialise value function
    while True:
        delta = 0
        for state in range(num_states):
            v = V[state]       # v is storing the value of previous value function
            if state==5:
                continue
            new_v = 0 # new_v is storing updated value function of the above state
            for action, action_prob in enumerate(policy[state]):  # action_prob is probability of performing particular action according to above policy
                for prob, next_state, reward, is_terminal in prob_dynamics[state][action]:
                    new_v += action_prob * prob * (reward + discount_factor * V[next_state])
            V[state] = new_v
            delta = max(delta, abs(v - V[state])) 
        if delta < theta:         # checking the convergence of the value function 
            break
    return V

# Run policy iteration
optimal_policy_PI, optimal_value_function,num_iteration_PI = policy_iteration(prob_dynamics, actions_dict)
print("Optimal Policy")
print(optimal_policy_PI)
print("number of iterations")
print(num_iteration_PI)
print("Optimal value function")
print(optimal_value_function)



Optimal Policy
[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
number of iterations
5
Optimal value function
[ 0.0839009   0.2806644   0.83816021  0.          0.01831314  0.
  0.16584507  0.         -0.00414617  0.00398712  0.04271207 -0.01295202]


In [213]:
# Running the optimal policy
is_Terminal = False
env.reset()
count = 0
total_reward = 0
state=0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    action = np.argmax(optimal_policy_PI[state])  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(action)
    
    total_reward += reward

    print("  ", rand_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)


Action	 State	 Reward	 is_Terminal
   2 	   0 	 -0.01 	   False
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 4
Final Reward: 0.97


## Q2. Find an optimal policy to navigate the given environment using Value Iteration

In [215]:
import numpy as np

# Define the Value Iteration function
def value_iteration(prob_dynamics, actions_dict, discount_factor=0.4, theta=0.0001,num_iteration=0):
    num_states = 12
    num_actions = 4
    
    # Initialize the value function
    V = np.zeros(num_states)
    # num_iteration =0
    
    while True:
        num_iteration+=1
        delta = 0  # Initialize the maximum change in value function
        for state in range(num_states):
            if state == 5:  # Skip terminal state
                continue
                
            v = V[state]  # Store the old value function
            
            # Compute the value for each action
            action_values = np.zeros(num_actions)
            for action in range(num_actions):
                for prob, next_state, reward, is_Terminal in prob_dynamics[state][action]:
                    action_values[action] += prob * (reward + discount_factor * V[next_state])
            
            # Update the value function to be the maximum action value
            V[state] = np.max(action_values)
            
            # Update the maximum change in value function
            delta = max(delta, np.abs(v - V[state]))
        
        # Check for convergence
        if delta < theta:
            break
            
    # Compute the optimal policy based on the computed value function
    policy = np.zeros((num_states, num_actions))
    for state in range(num_states):
        if state == 5:  # Skip terminal state
            continue
            
        # Compute action values for the current state
        action_values = np.zeros(num_actions)
        for action in range(num_actions):
            for prob, next_state, reward, is_Terminal in prob_dynamics[state][action]:
                action_values[action] += prob * (reward + discount_factor * V[next_state])
        
        # Choose the action with the maximum action value
        best_action = np.argmax(action_values)
        policy[state, best_action] = 1  # Set the probability of the best action to 1
        
    return policy, V, num_iteration

optimal_policy_VI, optimal_value_function,num_iteration_VI = value_iteration(prob_dynamics, actions_dict)

print("Optimal Policy")
print(optimal_policy_VI)
print("number of iteration")
print(num_iteration_VI)
print("Optimal value function")
print(optimal_value_function)




Optimal Policy
[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
number of iteration
7
Optimal value function
[ 0.0838983   0.28066419  0.83816021  0.          0.01831065  0.
  0.16584507  0.         -0.00414728  0.00398788  0.04271279 -0.01294585]


## Q3. Compare PI and VI in terms of convergence. Is the policy obtained by both same?

### Answer: Convergence:

Above, We can see that PI converges in fewer iterations compared to VI because it explicitly alternates between policy evaluation and policy improvement. However, each iteration of PI may require more computation due to the separate policy evaluation step.

Yes, Policy obtained by both are same
